# 迴歸分析 (Regression)

## 學習目標
- 理解迴歸問題的定義與應用
- 掌握特徵工程技巧
- 學會模型選擇與超參數調優
- 實作 COVID-19 確診數預測（李宏毅 HW1）

## 對應課程
- [李宏毅 ML 2021 - Lecture 1: Regression](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW1: COVID-19 Cases Prediction](https://github.com/ga642381/ML2021-Spring/tree/main/HW01)

## 迴歸問題概述

```
迴歸 vs 分類
├── 迴歸 (Regression): 輸出連續值
│   例: 房價預測、溫度預測、銷量預測
│
└── 分類 (Classification): 輸出離散類別
    例: 垃圾郵件檢測、圖片分類
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

---
## Part 1: 線性迴歸基礎

### 數學表示

線性迴歸模型：
$$\hat{y} = w_1 x_1 + w_2 x_2 + ... + w_n x_n + b = \mathbf{w}^T \mathbf{x} + b$$

損失函數（MSE）：
$$L = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

In [ ]:
# ========== 從零實現線性迴歸 ==========

# 生成範例資料
np.random.seed(42)
X = np.random.randn(100, 1) * 2
y = 3 * X + 2 + np.random.randn(100, 1) * 0.5  # y = 3x + 2 + noise

# 轉為 Tensor
X_tensor = torch.FloatTensor(X)
y_tensor = torch.FloatTensor(y)

# 初始化參數
w = torch.randn(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 訓練
lr = 0.01
epochs = 100

losses = []
for epoch in range(epochs):
    # 前向傳播
    y_pred = X_tensor * w + b
    
    # 計算 MSE 損失
    loss = ((y_pred - y_tensor) ** 2).mean()
    losses.append(loss.item())
    
    # 反向傳播
    loss.backward()
    
    # 更新參數（手動梯度下降）
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad
        
        # 清零梯度
        w.grad.zero_()
        b.grad.zero_()

print(f"學習到的參數: w = {w.item():.4f}, b = {b.item():.4f}")
print(f"真實參數: w = 3, b = 2")

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].scatter(X, y, alpha=0.5, label='資料')
axes[0].plot(X, X_tensor.detach().numpy() * w.item() + b.item(), 'r-', label='預測線')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')
axes[0].legend()
axes[0].set_title('線性迴歸擬合')

axes[1].plot(losses)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MSE Loss')
axes[1].set_title('訓練損失')

plt.tight_layout()
plt.show()

---
## Part 2: 深度神經網路迴歸

對於複雜的非線性關係，我們使用深度神經網路。

In [ ]:
# ========== DNN 迴歸模型 ==========

class DNNRegressor(nn.Module):
    """
    深度神經網路迴歸模型
    
    適用於 COVID-19 預測等複雜迴歸任務
    """
    
    def __init__(self, input_dim, hidden_dims=[64, 32, 16], dropout=0.2):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
            ])
            prev_dim = hidden_dim
        
        # 輸出層（迴歸輸出 1 個值）
        layers.append(nn.Linear(prev_dim, 1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x).squeeze(-1)

# 測試模型
model = DNNRegressor(input_dim=10, hidden_dims=[64, 32, 16])
print(model)

x = torch.randn(4, 10)
out = model(x)
print(f"\n輸入: {x.shape}")
print(f"輸出: {out.shape}")

---
## Part 3: COVID-19 預測實戰（李宏毅 HW1）

### 任務說明

根據前幾天的 COVID-19 相關特徵，預測未來的確診數。

**特徵包括**：
- 各州的 COVID 相關統計（確診、死亡、檢測陽性率等）
- 人口統計特徵
- 行為指標（配戴口罩比例等）

**目標**：預測第 N+1 天的確診數

In [ ]:
# ========== 模擬 COVID-19 資料集 ==========

def generate_covid_like_data(n_samples=3000, n_features=93, n_days=5):
    """
    生成類似 COVID-19 HW1 的資料
    
    實際 HW1 資料結構:
    - 每個樣本有 93 個特徵
    - 包含 5 天的歷史數據
    - 目標是預測下一天的確診數
    """
    np.random.seed(42)
    
    # 模擬特徵
    # 特徵類型: 州編碼(one-hot), 確診數, 死亡數, 陽性率等
    X = np.random.randn(n_samples, n_features)
    
    # 模擬目標（使用部分特徵的非線性組合）
    # 實際上確診數與前幾天的確診數高度相關
    y = (
        0.5 * X[:, 0] +
        0.3 * X[:, 1] ** 2 +
        0.2 * np.sin(X[:, 2]) +
        0.1 * X[:, 3] * X[:, 4] +
        np.random.randn(n_samples) * 0.1
    )
    
    return X.astype(np.float32), y.astype(np.float32)

# 生成資料
X, y = generate_covid_like_data(n_samples=3000, n_features=93)
print(f"特徵維度: {X.shape}")
print(f"標籤維度: {y.shape}")
print(f"標籤範圍: [{y.min():.2f}, {y.max():.2f}]")

In [ ]:
# ========== Dataset 類別 ==========

class COVID19Dataset(Dataset):
    """
    COVID-19 預測資料集
    """
    
    def __init__(self, X, y=None, mode='train'):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y) if y is not None else None
        self.mode = mode
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# 分割資料
train_ratio = 0.8
n_train = int(len(X) * train_ratio)

X_train, X_val = X[:n_train], X[n_train:]
y_train, y_val = y[:n_train], y[n_train:]

print(f"訓練集: {X_train.shape[0]} 樣本")
print(f"驗證集: {X_val.shape[0]} 樣本")

In [ ]:
# ========== 特徵工程 ==========

class FeatureEngineer:
    """
    特徵工程類別
    
    包含:
    - 特徵標準化
    - 特徵選擇
    - 缺失值處理
    """
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.selected_features = None
    
    def fit(self, X, y=None):
        """擬合標準化器"""
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        """轉換特徵"""
        X_scaled = self.scaler.transform(X)
        return X_scaled
    
    def fit_transform(self, X, y=None):
        """擬合並轉換"""
        self.fit(X, y)
        return self.transform(X)
    
    @staticmethod
    def select_features(X, y, method='correlation', k=40):
        """
        特徵選擇
        
        李宏毅 HW1 技巧:
        - 選擇與目標相關性高的特徵
        - 移除冗餘特徵
        """
        if method == 'correlation':
            correlations = []
            for i in range(X.shape[1]):
                corr = np.corrcoef(X[:, i], y)[0, 1]
                correlations.append(abs(corr) if not np.isnan(corr) else 0)
            
            # 選擇相關性最高的 k 個特徵
            top_indices = np.argsort(correlations)[-k:]
            return sorted(top_indices)
        
        return list(range(X.shape[1]))

# 應用特徵工程
fe = FeatureEngineer()

# 標準化
X_train_scaled = fe.fit_transform(X_train)
X_val_scaled = fe.transform(X_val)

# 特徵選擇（選擇最相關的 40 個特徵）
selected_idx = FeatureEngineer.select_features(X_train_scaled, y_train, k=40)
print(f"選擇的特徵數: {len(selected_idx)}")

X_train_selected = X_train_scaled[:, selected_idx]
X_val_selected = X_val_scaled[:, selected_idx]

print(f"特徵選擇後維度: {X_train_selected.shape}")

In [ ]:
# ========== 訓練配置 ==========

config = {
    'batch_size': 256,
    'epochs': 100,
    'lr': 1e-3,
    'weight_decay': 1e-5,
    'hidden_dims': [64, 32, 16],
    'dropout': 0.2,
    'early_stopping_patience': 10,
}

# 建立 DataLoader
train_dataset = COVID19Dataset(X_train_selected, y_train)
val_dataset = COVID19Dataset(X_val_selected, y_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

print(f"訓練批次數: {len(train_loader)}")
print(f"驗證批次數: {len(val_loader)}")

In [ ]:
# ========== 訓練函數 ==========

def train_regression_model(model, train_loader, val_loader, config):
    """
    訓練迴歸模型
    
    包含:
    - Early Stopping
    - 學習率調度
    - 最佳模型保存
    """
    model = model.to(device)
    
    # 優化器
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=config['lr'],
        weight_decay=config['weight_decay']
    )
    
    # 學習率調度
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # 損失函數
    criterion = nn.MSELoss()
    
    # 歷史記錄
    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(config['epochs']):
        # ========== 訓練 ==========
        model.train()
        train_loss = 0
        
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * len(X_batch)
        
        train_loss /= len(train_loader.dataset)
        
        # ========== 驗證 ==========
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                
                pred = model(X_batch)
                loss = criterion(pred, y_batch)
                
                val_loss += loss.item() * len(X_batch)
        
        val_loss /= len(val_loader.dataset)
        
        # 更新學習率
        scheduler.step(val_loss)
        
        # 記錄
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        # Early Stopping 檢查
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{config['epochs']}")
            print(f"  Train Loss: {train_loss:.6f}")
            print(f"  Val Loss: {val_loss:.6f}")
            print(f"  Best Val Loss: {best_val_loss:.6f}")
        
        if patience_counter >= config['early_stopping_patience']:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    # 載入最佳模型
    model.load_state_dict(best_model_state)
    
    return model, history

# 訓練模型
model = DNNRegressor(
    input_dim=X_train_selected.shape[1],
    hidden_dims=config['hidden_dims'],
    dropout=config['dropout']
)

model, history = train_regression_model(model, train_loader, val_loader, config)

In [ ]:
# ========== 訓練結果可視化 ==========

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 損失曲線
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Validation')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training History')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 預測 vs 實際
model.eval()
with torch.no_grad():
    X_val_tensor = torch.FloatTensor(X_val_selected).to(device)
    y_pred = model(X_val_tensor).cpu().numpy()

axes[1].scatter(y_val, y_pred, alpha=0.5)
axes[1].plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', label='Perfect')
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
axes[1].set_title('Prediction vs Actual')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 計算指標
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"\n評估指標:")
print(f"  MSE: {mse:.6f}")
print(f"  RMSE: {rmse:.6f}")
print(f"  MAE: {mae:.6f}")
print(f"  R²: {r2:.6f}")

---
## Part 4: 進階技巧

### 4.1 K-Fold 交叉驗證

In [ ]:
# ========== K-Fold 交叉驗證 ==========

def cross_validate(X, y, n_folds=5, config=None):
    """
    K-Fold 交叉驗證
    
    更可靠的模型評估方式
    """
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
        print(f"\n=== Fold {fold+1}/{n_folds} ===")
        
        # 分割資料
        X_tr, X_vl = X[train_idx], X[val_idx]
        y_tr, y_vl = y[train_idx], y[val_idx]
        
        # 標準化
        scaler = StandardScaler()
        X_tr = scaler.fit_transform(X_tr)
        X_vl = scaler.transform(X_vl)
        
        # 建立 DataLoader
        train_ds = COVID19Dataset(X_tr, y_tr)
        val_ds = COVID19Dataset(X_vl, y_vl)
        
        train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
        val_dl = DataLoader(val_ds, batch_size=256, shuffle=False)
        
        # 訓練
        model = DNNRegressor(input_dim=X.shape[1], hidden_dims=[64, 32], dropout=0.2)
        
        # 簡化訓練
        model = model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        
        for epoch in range(30):
            model.train()
            for X_batch, y_batch in train_dl:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()
        
        # 評估
        model.eval()
        with torch.no_grad():
            X_vl_tensor = torch.FloatTensor(X_vl).to(device)
            y_pred = model(X_vl_tensor).cpu().numpy()
        
        mse = mean_squared_error(y_vl, y_pred)
        fold_results.append(mse)
        print(f"  Fold MSE: {mse:.6f}")
    
    print(f"\n平均 MSE: {np.mean(fold_results):.6f} ± {np.std(fold_results):.6f}")
    return fold_results

# 執行交叉驗證
cv_results = cross_validate(X, y, n_folds=5)

### 4.2 正則化技巧

In [ ]:
# ========== 正則化技巧 ==========

class RegularizedRegressor(nn.Module):
    """
    帶正則化的迴歸模型
    
    包含多種正則化技巧：
    - L2 正則化 (weight_decay)
    - Dropout
    - Batch Normalization
    - Early Stopping
    """
    
    def __init__(self, input_dim, hidden_dims, dropout=0.3):
        super().__init__()
        
        layers = []
        prev_dim = input_dim
        
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.LeakyReLU(0.1))  # LeakyReLU 防止死神經元
            
            # 漸進式 dropout（前面層少，後面層多）
            layers.append(nn.Dropout(dropout * (i + 1) / len(hidden_dims)))
            
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x).squeeze(-1)

print("正則化迴歸模型：")
reg_model = RegularizedRegressor(input_dim=40, hidden_dims=[64, 32, 16], dropout=0.3)
print(reg_model)

---
## 🎯 總結

### 迴歸任務重點

| 技巧 | 說明 |
|------|------|
| **特徵標準化** | 加速收斂，必做 |
| **特徵選擇** | 移除無關特徵，減少過擬合 |
| **Early Stopping** | 防止過擬合 |
| **Learning Rate Scheduling** | 後期精細調整 |
| **Cross Validation** | 更可靠的評估 |

### 評估指標

| 指標 | 公式 | 說明 |
|------|------|------|
| **MSE** | $\frac{1}{n}\sum(y-\hat{y})^2$ | 均方誤差，對大誤差敏感 |
| **RMSE** | $\sqrt{MSE}$ | 與原始尺度相同 |
| **MAE** | $\frac{1}{n}\sum|y-\hat{y}|$ | 平均絕對誤差 |
| **R²** | $1 - \frac{SS_{res}}{SS_{tot}}$ | 決定係數，越接近 1 越好 |

### 李宏毅 HW1 技巧

```
Strong Baseline 達成方法:
1. 特徵選擇：選擇與確診數相關的特徵
2. 適當的網路深度：不要太深也不要太淺
3. 正則化：Dropout + Weight Decay
4. 早停：監控驗證集損失
5. 學習率調度：ReduceLROnPlateau
```

### 下一步

前往 `supervised_learning/classification.ipynb` 學習分類任務！

---
## 練習題

### 練習 1: 比較不同特徵數量的效果

**目標**: 實驗不同特徵選擇數量對模型性能的影響

In [ ]:
# 練習 1: 特徵選擇實驗

def experiment_feature_selection(X_train, y_train, X_val, y_val, k_values=[10, 20, 40, 60, 80]):
    """比較不同特徵數量的效果"""
    results = []
    
    # 標準化
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    for k in k_values:
        # 特徵選擇
        selected_idx = FeatureEngineer.select_features(X_train_scaled, y_train, k=min(k, X_train.shape[1]))
        X_tr_sel = X_train_scaled[:, selected_idx]
        X_vl_sel = X_val_scaled[:, selected_idx]
        
        # 訓練模型
        model = DNNRegressor(input_dim=len(selected_idx), hidden_dims=[32, 16], dropout=0.2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        
        train_ds = COVID19Dataset(X_tr_sel, y_train)
        train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)
        
        for epoch in range(50):
            model.train()
            for X_batch, y_batch in train_dl:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                loss = criterion(model(X_batch), y_batch)
                loss.backward()
                optimizer.step()
        
        # 評估
        model.eval()
        with torch.no_grad():
            y_pred = model(torch.FloatTensor(X_vl_sel).to(device)).cpu().numpy()
        
        mse = mean_squared_error(y_val, y_pred)
        results.append({'k': k, 'mse': mse})
        print(f"k={k}: MSE = {mse:.6f}")
    
    return results

# 執行實驗
results = experiment_feature_selection(X_train, y_train, X_val, y_val)

# 視覺化
plt.figure(figsize=(8, 5))
plt.plot([r['k'] for r in results], [r['mse'] for r in results], 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Features (k)')
plt.ylabel('Validation MSE')
plt.title('Feature Selection: MSE vs Number of Features')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n最佳特徵數: k={min(results, key=lambda x: x['mse'])['k']}")

### 練習 2: 比較不同損失函數

**目標**: 比較 MSE、MAE、Huber Loss 對異常值的敏感度

In [ ]:
# 練習 2: 比較損失函數

# 生成帶異常值的資料
np.random.seed(42)
X_clean = np.random.randn(200, 5).astype(np.float32)
y_clean = (2 * X_clean[:, 0] + 3 * X_clean[:, 1] + np.random.randn(200) * 0.3).astype(np.float32)

# 加入 10% 異常值
outlier_idx = np.random.choice(200, 20, replace=False)
y_with_outliers = y_clean.copy()
y_with_outliers[outlier_idx] += np.random.choice([-10, 10], 20)

print(f"異常值數量: {len(outlier_idx)}")
print(f"正常 y 範圍: [{y_clean.min():.2f}, {y_clean.max():.2f}]")
print(f"異常 y 範圍: [{y_with_outliers.min():.2f}, {y_with_outliers.max():.2f}]")

def train_with_loss(X, y, loss_fn, loss_name, epochs=100):
    """使用指定損失函數訓練"""
    model = nn.Sequential(nn.Linear(5, 32), nn.ReLU(), nn.Linear(32, 1)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    X_t = torch.FloatTensor(X).to(device)
    y_t = torch.FloatTensor(y).to(device)
    
    for _ in range(epochs):
        optimizer.zero_grad()
        pred = model(X_t).squeeze()
        loss = loss_fn(pred, y_t)
        loss.backward()
        optimizer.step()
    
    with torch.no_grad():
        pred = model(X_t).squeeze().cpu().numpy()
    
    # 計算在乾淨資料上的 MSE
    mse_clean = mean_squared_error(y_clean, pred)
    return mse_clean, pred

# 比較三種損失函數
loss_fns = {
    'MSE': nn.MSELoss(),
    'MAE (L1)': nn.L1Loss(),
    'Huber': nn.HuberLoss(delta=1.0),
}

results_losses = {}
for name, loss_fn in loss_fns.items():
    mse, pred = train_with_loss(X_clean, y_with_outliers, loss_fn, name)
    results_losses[name] = {'mse': mse, 'pred': pred}
    print(f"{name}: Clean MSE = {mse:.4f}")

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for i, (name, res) in enumerate(results_losses.items()):
    axes[i].scatter(y_clean, res['pred'], alpha=0.5)
    axes[i].plot([y_clean.min(), y_clean.max()], [y_clean.min(), y_clean.max()], 'r--')
    axes[i].set_xlabel('True y (clean)')
    axes[i].set_ylabel('Predicted y')
    axes[i].set_title(f"{name}\nMSE on clean data: {res['mse']:.4f}")
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Loss Function Comparison with Outliers in Training Data')
plt.tight_layout()
plt.show()

print("\n結論: MAE 和 Huber Loss 對異常值更魯棒")